In [2]:
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from label_ranking import *
from massart import *

N = 20000
d = 10  # data dimension
k = 5  # labels
epsilon = 0.3  # upper bound for classification error
delta = 0.4  # 1 - delta is the confidence
eta_max = 0.1

D = multivariate_normal(mean=np.zeros(d), cov=np.identity(d))  # distribution

W_opt = np.random.rand(k, d)

X_train = D.rvs(size=N)

R_train_ground_true = ground_truth_permutations(X_train, W_opt)
P_train_ground_true = np.array([np.argsort(ranking) for ranking in R_train_ground_true])

R_train = np.array([addNoise(y, eta_max) for y in R_train_ground_true])
P_train = np.array([np.argsort(ranking) for ranking in R_train])

In [3]:
P_train.shape[1]

5

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

DTR = [DecisionTreeRegressor().fit(X_train, P_train[:, label]) for label in range(k)]
RFR = [RandomForestRegressor().fit(X_train, P_train[:, label]) for label in range(k)]

In [5]:
X_test = D.rvs(size=10000)

R_test_ground_true = ground_truth_permutations(X_test, W_opt)
P_test_ground_true = np.array([np.argsort(ranking) for ranking in R_test_ground_true])

In [6]:
Y_pred_DTR = np.array([DTR[label].predict(X_test) for label in range(k)]).T
Y_pred_RFR = np.array([RFR[label].predict(X_test) for label in range(k)]).T

In [7]:
R_pred_DTR = np.argsort(Y_pred_DTR, axis=1)
R_pred_RFR = np.argsort(Y_pred_RFR, axis=1)

In [8]:
# P_pred_DTR = np.array([np.argsort(ranking) for ranking in R_pred_DTR])
# P_pred_RFR = np.array([np.argsort(ranking) for ranking in R_pred_RFR])

P_pred_DTR = np.argsort(R_pred_DTR, axis=1)
P_pred_RFR = np.argsort(R_pred_RFR, axis=1)

In [9]:
P_pred_DTR

array([[4, 1, 3, 0, 2],
       [2, 0, 3, 4, 1],
       [4, 2, 0, 1, 3],
       ...,
       [1, 2, 3, 4, 0],
       [1, 3, 2, 0, 4],
       [2, 0, 1, 3, 4]], dtype=int64)

In [10]:
def acc(p1, p2):
  assert len(p1) == len(p2)
  k = len(p1)
  goodPairs = 0
  for (i, j) in combinations(range(k), 2):
    if (p1[i] - p1[j])*(p2[i] - p2[j]) > 0: goodPairs += 1
  return 2*goodPairs/k/(k-1)

In [11]:
from scipy.stats import kendalltau

In [12]:
avg_KTdist_DTR = np.mean([acc(P_pred_DTR[i], P_test_ground_true[i]) for i in range(len(P_pred_DTR))])
avg_KTdist_RFR = np.mean([acc(P_pred_RFR[i], P_test_ground_true[i]) for i in range(len(P_pred_RFR))])

avg_KTdist_DTR2 = np.mean([acc(P_pred_DTR[i], P_test_ground_true[i]) for i in range(len(P_pred_DTR))])
avg_KTdist_RFR2 = np.mean([acc(P_pred_RFR[i], P_test_ground_true[i]) for i in range(len(P_pred_RFR))])

print("Decision tree regressor average normalized KT distance:", avg_KTdist_DTR)
print("Random forest regressor average normalized KT distance:", avg_KTdist_RFR)

Decision tree regressor average normalized KT distance: 0.7986000000000001
Random forest regressor average normalized KT distance: 0.9304


In [13]:
from lr_decision_trees import Regressor_KTDistance

In [14]:
SC = Regressor_KTDistance(X_train, P_train, X_test, P_test_ground_true)
# avg_KTdist_DTR = np.mean([acc(pr[i], P_test_ground_true[i]) for i in range(len(pr))])
print("Decision tree regressor average normalized KT distance:", SC)

Decision tree regressor average normalized KT distance: 0.8004800000000001


In [15]:
# Regressor_KTDistance(X_train, P_train, X_test, P_test_ground_true, RandomForestRegressor())